In [2]:
include("./mm.jl");

WebIO._IJuliaInit()

In [2]:
# s is the stock prices. it is the output variable. 
# ϕ is a temporary array which will be used to hold the normal distribution
function weiner(days::Int, μ::Float64, σ::Float64, s::Vector{Float64}, 
        ϕ::Vector{Float64}, dist::Normal{Float64})
    s[1] = 100
    a::Float64 = μ/365.0
    b::Float64 = σ/sqrt(365.0)
    rand!(dist, ϕ)
    for i::Int in 2:days
        s[i] = s[i-1]*(1 + a + ϕ[i]*b)
    end  
    return s
end

function downscale(Y::Vector{Float64}, points::Int)
    days::Int = length(Y)
    step::Float64 = 1
    if days > points
        step = days / points
    end
    println(step)
    XX = Vector{Float64}(undef, 0)
    YY = Vector{Float64}(undef, 0)
    for i in 1:step:days
        push!(XX, Int(i))
        push!(YY, Y[min(days, Int(round(i)))])
    end
    return XX, YY
end
days = 10000
μ, σ = 10.0/100, 30.0/100
# X, Y = downscale(weiner(days, μ, σ), 5000)
# doplot(X, Y)


(0.1, 0.3)

In [1]:
# Strategy: buy a stock every day, sell it when it hits cagr of 10%. or a maximum of 1000 days
function simulateW()
    runs::Int = 5000
    wallets::Vector{Float64} = Vector{Float64}(undef, runs)
    
    rateThreshold::Float64 = .1; investDays::Int = 365*3; redeemDays::Int=365; days = investDays + redeemDays; 
    μ::Float64=.1; σ::Float64=.3; ll = FastList{Tuple{Int, Float64}}(4000);
    s::Union{Vector{Float64}, Nothing}=Vector{Float64}(undef, days); 
        ϕ::Union{Vector{Float64}, Nothing}=Vector{Float64}(undef, days); dist::Union{Normal{Float64}, Nothing}=Normal(0, 1);
    
    for run::Int in 1:runs
        wallets[run] = oneRun(rateThreshold, investDays, redeemDays, μ, σ, ll, s, ϕ, dist)
        reset!(ll)
    end
    histogram(wallets)
    
end

function oneRun(rateThreshold::Float64, investDays::Int, redeemDays::Int, μ::Float64, σ::Float64, ll::FastList{Tuple{Int, Float64}}, 
        s::Union{Vector{Float64}, Nothing}=nothing, 
        ϕ::Union{Vector{Float64}, Nothing}=nothing, dist::Union{Normal{Float64}, Nothing}=nothing)
    days::Int = investDays+redeemDays
    s = weiner(days, μ, σ, s, ϕ, dist)
#     daysToRedeem = Vector{Int}(undef, days)
#     fill!(daysToRedeem, 0)
#     display(doplot(collect(1:days), s))
    
    minWallet = 0
    wallet::Float64 = 0
    for day::Int in 1:days
        for node in ll
            (d::Int, price::Float64) = node.data
            rate::Float64 = r(price, s[day], (day-d)/365.0)
            if rate > rateThreshold
                wallet += s[day]
                del(ll, node)
#                 daysToRedeem[d] = day-d
            end
        end
        if day <= investDays
            wallet -= s[day]
            push!(ll, (day, s[day]))
        end
        minWallet = min(wallet, minWallet)
    end
    # dispose all the stocks
    wallet += s[end]*ll.len
#     display(plot(fit(Histogram, daysToRedeem, 0:10)))
    return wallet
end
ll = FastList{Tuple{Int, Float64}}(4000)
# oneRun(0.1, 3, 3, .1, .2, ll)
@time simulate()

LoadError: UndefVarError: FastList not defined

In [ ]:
# Strategy: invest the same amount every day, sell it when it hits cagr of 10%. or a maximum of 1000 days
function simulateWeiner()
    runs::Int = 5000
    wallets::Vector{Float64} = Vector{Float64}(undef, runs)
    investDays::Int = 365*3; redeemDays::Int=365; days = investDays + redeemDays; 
    ll = FastList{Tuple{Int, Float64, Float64}}(investDays+redeemDays+1000);
    
    
    rateThreshold::Float64 = .1; 
    μ=.1; σ=.3; prices=Vector{Float64}(undef, days); ϕ=Vector{Float64}(undef, days); dist=Normal(0, 1);
    tradingDays = collect(1:days)
    for run::Int in 1:runs
        s = weiner(days, μ, σ, prices, ϕ, dist)
        wallets[run] = oneRun(rateThreshold, investDays, redeemDays,ll, prices)
        reset!(ll)
    end
    #histogram(wallets)
    
end

function oneRun(rateThreshold::Float64, investDays::Int, redeemDays::Int, ll::FastList{Tuple{Int, Float64, Float64}}, 
        s::Union{Vector{Float64}, Nothing}=nothing)
    days::Int = investDays+redeemDays
    amount::Int = 100
    
    minWallet = 0
    wallet::Float64 = 0
    for day::Int in 1:days
        for node in ll
            (d::Int, price::Float64, num::Float64) = node.data
            rate::Float64 = r(price, s[day], (day-d)/365.0)
            if rate > rateThreshold
                wallet += s[day]*num
                del(ll, node)
#                 daysToRedeem[d] = day-d
            end
        end
        if day <= investDays
            wallet -= amount
            push!(ll, (day, s[day], amount/s[day]))
        end
        minWallet = min(wallet, minWallet)
    end
    # dispose all the stocks
    for node in ll
        (d::Int, price::Float64, num::Float64) = node.data
        wallet += s[end]*num
    end
#     display(plot(fit(Histogram, daysToRedeem, 0:10)))
    return wallet
end
ll = FastList{Tuple{Int, Float64}}(4000)
# oneRun(0.1, 3, 3, .1, .2, ll)
@time simulate()

In [1]:
# this code is a backup copy of a working code. Please dont modify this. Rather, copy past it somewhere else and modify
io = initLogging()

function oneRun(rateThreshold::Float64, investDays::Int, redeemDays::Int, ll::FastList{Int}, 
        tradingDays::AbstractVector{Int}, prices::AbstractVector{Float64})
    days::Int = investDays+redeemDays
    wallet::Float64 = 0.0;
    amount = 1000.0;
    
    maxDaysHeld::Int = 0;
    minWallet = 0;
    rateGeom::Float64 = 1.0;
    totDaysHeld::Float64 = 0
    reset!(ll)
    
    for i::Int in 1:days
        for node in ll
            dayBought::Int = node.data
            daysHeld::Int = tradingDays[i] - tradingDays[dayBought]
            net_today::Float64 = net_today_f(amount, prices[dayBought], prices[i], daysHeld)
            rate::Float64 = r(amount, net_today, daysHeld/365.0);
            if rate > rateThreshold || i==days
                wallet += net_today
                maxDaysHeld = max(maxDaysHeld, daysHeld);
                totDaysHeld += daysHeld;
                rateGeom = rateGeom * (1+rate)^(daysHeld/365);
                del(ll, node)                
            end
        end
        if i <= investDays
            wallet -= amount
            push!(ll, i)
        end
        minWallet = min(wallet, minWallet)

    end
    return rateGeom^(365.0/totDaysHeld) - 1, wallet;
end

# tls is thread local storage
function simulateMulti(tls::Vector{Tls}, d::Dict{String, Point}, investDays::Int, redeemDays::Int)
    runs::Int = 5000
    
    days = investDays + redeemDays; 
    results = Vector{Float64}(undef, runs)
      
    rateThreshold::Float64 = .1; μ=meanNifty; σ=stdNifty;
    
    prices=d["^NSEI"].prices;
    tradingDays = d["^NSEI"].intDates;
    
    wallets = Vector{Float64}(undef, 0)
    bad_days = Vector{Int}(undef, 0)
    lk = Threads.SpinLock()
    
    @views Threads.@threads  for start in 1:length(tradingDays) - days - 10
#     @views Threads.@threads  for start in 1626:1626
        myTls::Tls = tls[Threads.threadid()]
        pprices = prices[start:end];
        ddates = tradingDays[start:end];

        rateGeom, wallet = oneRun(rateThreshold, investDays, redeemDays, myTls.ll, ddates, pprices);
        lock(lk) do
            push!(wallets, rateGeom);
            if wallet < 0
                push!(bad_days, start)
            end
        end
    end
    display( plot([histogram(x=wallets, nbins=100)]))
    println("mean is $(mean(wallets)), stdev is $(std(wallets))")
end


#     sort!(negativeDays)
#     println(negativeDays)
#     y = nquantile(wallets, 100)
#     x = collect(1:101)
#     doplot(x, y)
#     dates, prices, intDates = d["^NSEI"].dates, d["^NSEI"].prices, d["^NSEI"].intDates
#     range = 1:length(dates)
#     trace1 = scatter(x=range, y=prices,
#                         mode="lines+markers",
#                         name="nifty 50")
#     trace2 = scatter(x=range[bad_days], y=prices[bad_days],
#                         mode="markers",
#                         name="start times where the returns are negative")
#     add = investDays + redeemDays
#     trace3 = scatter(x=range[bad_days.+investDays], y=prices[bad_days.+investDays],
#                         mode="markers",
#         name="invest time of the start times")
#     trace4 = scatter(x=range[bad_days.+add], y=prices[bad_days.+add],
#                         mode="markers",
#         name="invest and redeem time of the start times")

#     trace5 = scatter(x=1815:(1815+2000), y=map(x->p2(8996, .1, x/365.0), 1:2000),
#                         mode="markers",
#         name="10% rate growth")

#     display(plot([trace1, trace2, trace3, trace4, trace5]))#     plot(1:101, nquantile(wallets, 100))
#fuck
# tls is thread local storage
function simulateWeinerMulti(tls::Vector{Tls})
    runs::Int = 1000
    investDays = 365*3; redeemDays=365; 
    
    days = investDays + redeemDays; 
    results = Vector{Float64}(undef, 0)
    
    
    rateThreshold::Float64 = .4; μ=.1; σ=.3;
    
    prices=Vector{Float64}(undef, days);
    tradingDays = collect(1:days)
    
    lk = Threads.SpinLock()

    Threads.@threads for run::Int in 1:runs
        tid = Threads.threadid()
        myTls = tls[tid]
        brk = false
        weiner(days, μ, σ, myTls.prices, myTls.ϕ, myTls.dist)
        reset!(myTls.ll)
        result = oneRun(rateThreshold, investDays, redeemDays, myTls.ll, tradingDays, myTls.prices)
        lock(lk) do
            push!(results, result)
        end
    end
    display( plot([histogram(x=results)]))
end
# @time simulateWeinerMulti(tls)
@time simulateMulti(tls, d, 365, 365*3)
# may 5th 2017 to nov 24 2017
flush(io)

LoadError: UndefVarError: initLogging not defined